In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta, date
import time
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import matplotlib.patches as patches
from matplotlib.colors import TwoSlopeNorm
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from scipy import linalg
from scipy import stats
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mapping_tickers import *
from mapping_plot_attributes import theme_style
from utils import *

### Testing On A Portfolio Of The Magnificent Seven

In [2]:
tickers = list(magnificent_7_tickers.keys())
end_date = datetime.today()
hist_years, hist_months, hist_days = 1, 0, 0
start_date = datetime(end_date.year - hist_years, end_date.month - hist_months, end_date.day - hist_days)

df_adj_close = pd.DataFrame()
for tk in tickers:
    data = yf.download(tk, start=start_date, end=end_date)
    df_adj_close[tk] = data['Adj Close']

df_adj_close = df_adj_close.dropna()

display(df_adj_close)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,NVDA,AAPL,TSLA,MSFT,AMZN,META,GOOG
Date,,,,,,,
2023-09-13,45.471245,173.323090,271.299988,333.556335,144.850006,304.434418,137.163162
2023-09-14,45.567219,174.845306,276.040009,336.176666,144.720001,311.080750,138.649506
2023-09-15,43.886723,174.119003,274.390015,327.759827,140.389999,299.694122,137.961197
2023-09-18,43.952698,177.063965,265.279999,326.608459,139.979996,301.929565,138.619583
2023-09-19,43.506836,178.158340,266.500000,326.201477,137.630005,304.444397,138.489899
...,...,...,...,...,...,...,...
2024-09-09,106.460892,220.910004,216.270004,405.720001,175.399994,504.790009,149.539993
2024-09-10,108.090752,220.110001,226.169998,414.200012,179.550003,504.790009,150.009995
2024-09-11,116.900002,222.660004,228.130005,423.040009,184.520004,511.829987,152.149994


In [3]:
def longest_drawdown(dd, tk):
    """
    dd:  series (column) of drawdowns for a ticker tk
    tk:  ticker
    returns dictionary longest_drawdown_data = {
        'Longest Drawdown': df_longest,
        'Total Length': total_length,
        'Length To Trough': length_to_trough,
        'Trough': trough,
        'Trough Date': trough_date
        }
    """

    dd_tk = pd.DataFrame(dd, columns=[tk], index=dd.index)
    df_longest = pd.DataFrame(columns=[tk])
       
    total_length = 0

    for i, idx in enumerate(dd_tk.index):

        # Is it a peak?
        if dd_tk.loc[idx, tk] == 0:
            # If so, let's mark it
            peak_idx = idx
            # Let's look at the subset from the peak to the end of the data
            sub_dd_tk = dd_tk.iloc[i + 1:]  # dd_tk[i + 1:] works too

            # Are there any zeros, i.e. other peaks, in this subset?
            if sub_dd_tk[tk].max() == 0:
                # If so, then take the earliest of them as a candidate to mark the recovery from this drawdown
                end_idx = sub_dd_tk.index[sub_dd_tk[tk] == 0].min()
            else:
                # If not, then it must be the last drawdown of the historical period, not yet recovered
                end_idx = sub_dd_tk.index[-1]

            # Does this subset have a negative minimum or is it just a flat segment of zeros?
            if dd_tk.loc[peak_idx: end_idx, tk].min() < 0:
                # If it does have a negative minimum, then is this drawdown the longest one so far?
                if len(dd_tk[peak_idx: end_idx]) > total_length:
                    # If so, then update the longest drawdown and its length
                    df_longest = dd_tk[peak_idx: end_idx].copy()
                    total_length = len(df_longest)

    trough = df_longest[tk].min()
    trough_datetime = df_longest.index[df_longest[tk] == trough][0]
    trough_date = trough_datetime.date()
    length_to_trough = len(df_longest[df_longest.index <= trough_datetime])

    longest_drawdown_data = {
        'Longest Drawdown': df_longest,
        'Total Length': total_length,
        'Length To Trough': length_to_trough,
        'Trough': trough,
        'Trough Date': trough_date
    }

    return longest_drawdown_data

In [4]:
def summarize_portfolio_drawdowns(df, tickers):
    """
    df:         input dataframe of historical prices, such as df_adj_close
    tickers:    Yahoo!Finance tickers in the portfolio
    return:     drawdown_data = {
                    'Drawdown Stats': df_drawdown_stats
                }
    """

    drawdown_metrics = [
        'Max Drawdown Trough (%)',
        'Max Drawdown Trough Date',
        'Max Drawdown Length (Days)',
        'Average Drawdown Trough (%)',
        'Longest Drawdown (Days)',
        'Longest Drawdown Trough (%)',
        'Longest Drawdown Trough Date',
        'Total Ulcer Index (%)',
        'Max 14-Day Ulcer Index (%)',
        'Max 14-Day Ulcer Date'
    ]
    df_drawdown_stats = pd.DataFrame(columns=tickers, index=drawdown_metrics).astype(str)

    df_drawdowns = pd.DataFrame(columns=tickers, index=df.index)
    returns = df / df.shift(1) - 1
    df_returns = pd.DataFrame(returns, columns=tickers, index=df.index).dropna()
    
    for tk in tickers:
        df_drawdowns.loc[df_drawdowns.index[0], tk] = 0

    for i, idx in enumerate(df_drawdowns.index[1:]):
        idx_prev = df_drawdowns.index[i]
        for tk in tickers:
            current_drawdown = (1 + df_returns.loc[idx, tk]) * (1 + df_drawdowns.loc[idx_prev, tk]) - 1
            df_drawdowns.loc[idx, tk] = np.min([current_drawdown, 0])

    df_drawdowns = df_drawdowns.astype(float) * 100
    max_index = max(df_drawdowns.index)
    print(max_index)

    df_drawdowns_squared = df_drawdowns * df_drawdowns
    n = len(df_drawdowns_squared)

    roll_ulcer = np.sqrt(df_drawdowns_squared.rolling(window=14, min_periods=1).sum() / 14)
    df_roll_ulcer = pd.DataFrame(roll_ulcer, columns=tickers)

    for tk in tickers:

        drawdowns_tk = df_drawdowns[tk]

        max_drawdown_trough = drawdowns_tk.min()
        max_drawdown_trough_datetime = df_drawdowns.index[drawdowns_tk == max_drawdown_trough][0]
        max_drawdown_trough_date = max_drawdown_trough_datetime.date()

        max_drawdown_peak = max(drawdowns_tk.index[(drawdowns_tk.index < max_drawdown_trough_datetime) & (drawdowns_tk == 0)])
        index_recovery = drawdowns_tk.index[(drawdowns_tk.index > max_drawdown_trough_datetime) & (drawdowns_tk == 0)]
        if len(index_recovery) == 0:
            max_drawdown_recovery = max_index
        else:
            max_drawdown_recovery = min(index_recovery)
        max_drawdown_length = len(drawdowns_tk[max_drawdown_peak: max_drawdown_recovery])
       
        df_drawdown_stats.loc['Max Drawdown Trough (%)', tk] = f'{drawdowns_tk.min():.2f}'
        df_drawdown_stats.loc['Max Drawdown Trough Date', tk] = f'{max_drawdown_trough_date}'
        df_drawdown_stats.loc['Max Drawdown Length (Days)', tk] = f'{max_drawdown_length}'
        df_drawdown_stats.loc['Average Drawdown Trough (%)', tk] = f'{drawdowns_tk.mean():.2f}'

        longest_drawdown_data = longest_drawdown(drawdowns_tk, tk)
        longest_drawdown_length = longest_drawdown_data['Total Length']
        longest_drawdown_trough = longest_drawdown_data['Trough']
        longest_drawdown_trough_date = longest_drawdown_data['Trough Date']
        df_drawdown_stats.loc['Longest Drawdown (Days)', tk] = f'{longest_drawdown_length}'
        df_drawdown_stats.loc['Longest Drawdown Trough (%)', tk] = f'{longest_drawdown_trough:.2f}'
        df_drawdown_stats.loc['Longest Drawdown Trough Date', tk] = f'{longest_drawdown_trough_date}'

        roll_ulcer_tk = df_roll_ulcer[tk]
        ulcer_index = np.sqrt(df_drawdowns_squared[tk].sum() / n)
        df_drawdown_stats.loc['Total Ulcer Index (%)', tk] = f'{ulcer_index:.2f}'
        max_14d_ulcer = roll_ulcer_tk.max()
        max_14d_ulcer_date = df_roll_ulcer.index[roll_ulcer_tk == max_14d_ulcer][0].date()
        df_drawdown_stats.loc['Max 14-Day Ulcer Index (%)', tk] = f'{max_14d_ulcer:.2f}'
        df_drawdown_stats.loc['Max 14-Day Ulcer Date', tk] = f'{max_14d_ulcer_date}'

    return df_drawdown_stats


dd = summarize_portfolio_drawdowns(df_adj_close, tickers)
display(dd)


2024-09-13 00:00:00


,NVDA,AAPL,TSLA,MSFT,AMZN,META,GOOG
Max Drawdown Trough (%),-27.05,-16.61,-48.54,-15.49,-19.49,-18.43,-22.28
Max Drawdown Trough Date,2024-08-07,2024-04-19,2024-04-22,2024-08-05,2024-08-05,2024-04-30,2024-09-09
Max Drawdown Length (Days),61,123,252,50,50,63,47
Average Drawdown Trough (%),-5.89,-5.27,-23.90,-3.56,-4.59,-4.42,-5.11
Longest Drawdown (Days),61,123,252,50,50,63,51
Longest Drawdown Trough (%),-27.05,-16.61,-48.54,-15.49,-19.49,-18.43,-12.91
Longest Drawdown Trough Date,2024-08-07,2024-04-19,2024-04-22,2024-08-05,2024-08-05,2024-04-30,2023-10-27
Total Ulcer Index (%),8.47,7.00,26.53,5.20,6.74,6.11,7.38
Max 14-Day Ulcer Index (%),20.76,14.52,41.93,12.35,15.16,13.96,17.92
Max 14-Day Ulcer Date,2024-08-12,2024-05-02,2024-04-24,2024-08-19,2024-08-21,2024-05-14,2024-09-13


In [14]:
def summarize_tk_drawdowns(df, tk, sort_by, n_top=5):
    """
    df:         series/dataframe of historical prices, such as df_adj_close
    tk:         ticker for which to perform the analysis
    sort_by:    column to sort by, should be a based on user input
    n_top:      number of top drawdowns to include in df_tk_deepest_drawdowns
    return:     drawdown_data = {
                    'Drawdown Stats': df_tk_drawdowns,
                    'Deepest Drawdowns': df_tk_deepest_drawdowns,
                    'Longest Drawdowns': df_tk_longest_drawdowns
                }
    """

    if isinstance(df, pd.Series):
        df_tk = df.copy()
    elif isinstance(df, pd.DataFrame):
        df_tk = df[tk]
    else:
        print('Incorrect format of input data')
        exit

    df_roll_max = pd.DataFrame(index=df_tk.index)
    drawdown_columns = [
        'Peak',
        'Trough',
        '% Drawdown',
        'Peak Date',
        'Trough Date',
        'Recovery Date',
        'Total Length',
        'Peak To Trough',
        'Trough To Recovery'
    ]
    cols_float = [
        'Peak',
        'Trough',
        '% Drawdown'
    ]
    cols_int = [
        'Total Length',
        'Peak To Trough',
        'Trough To Recovery'
    ]
    cols_str = [
        'Peak Date',
        'Trough Date',
        'Recovery Date'
    ]
    
    df_tk_drawdowns = pd.DataFrame(columns=drawdown_columns)
    df_tk_deepest_drawdowns = pd.DataFrame(columns=drawdown_columns)
    df_tk_longest_drawdowns = pd.DataFrame(columns=drawdown_columns)
    df_tk_drawdowns_str = pd.DataFrame(columns=drawdown_columns)
    df_tk_deepest_drawdowns_str = pd.DataFrame(columns=drawdown_columns)
    df_tk_longest_drawdowns_str = pd.DataFrame(columns=drawdown_columns)
    
    n = len(df)
    df_roll_max[tk] = df_tk.rolling(n, min_periods=1).max()
    unique_max_list = df_roll_max[tk].unique()
    
    for peak in unique_max_list:
    
        # Define a segment corresponding to vmax 
        cond = df_roll_max[tk] == peak
        seg = df_roll_max.loc[cond, tk]
        n_seg = len(seg)

        # There was no drop within a segment if its length is 1 or 2
        if n_seg > 2:

            # The first date of the segment (min_date_seg) may not necessarily be the first date of the drawdown; e.g. 
            # if the segment starts with a flat section. In that case the last date of the flat part becomes the min_date.

            min_date_seg = seg.index.min()
            max_date = seg.index.max()
            max_iloc = df.index.get_loc(max_date)

            cond_below_max = df_tk < peak
            cond_in_range = (df.index >= min_date_seg) & (df.index <= max_date) & cond_below_max

            min_date = df.loc[cond_in_range].index.min()
            min_iloc = df.index.get_loc(min_date)
            peak_date = min_date if min_iloc == 0 else df.index[min_iloc - 1]
            
            # trough = df.loc[cond_in_range, tk].min()
            trough = df_tk[cond_in_range].min()
            cond_trough = cond_in_range & (df_tk == trough)
            trough_date = df[cond_trough].index[0]
            recovery_date = max_date if max_iloc == n - 1 else df.index[max_iloc + 1]

            # NOTE: If the last Adj Close in the last segment is less than that segment's rolling max, then 
            # there was no recovery in that segment. It should still be reported if the drawdown in that segment is
            # significant, must marked somehow to indicate no recovery (e.g. as 0 or -1)

            cond_to_trough = (df.index >= min_date) & (df.index <= trough_date) & cond_below_max
            cond_recovery = (df.index > trough_date) & (df.index <= max_date) & cond_below_max
            n_to_trough = len(seg[cond_to_trough]) + 1
            n_recovery = len(seg[cond_recovery]) + 1
            n_length = n_to_trough + n_recovery
            drawdown = trough / peak - 1

            df_tk_drawdowns.loc[peak, 'Peak'] = peak
            df_tk_drawdowns.loc[peak, 'Trough'] = trough
            df_tk_drawdowns.loc[peak, 'Peak Date'] = peak_date
            df_tk_drawdowns.loc[peak, 'Trough Date'] = trough_date
            df_tk_drawdowns.loc[peak, 'Recovery Date'] = recovery_date
            df_tk_drawdowns.loc[peak, '% Drawdown'] = 100 * drawdown
            df_tk_drawdowns.loc[peak, 'Total Length'] = n_length
            df_tk_drawdowns.loc[peak, 'Peak To Trough'] = n_to_trough
            df_tk_drawdowns.loc[peak, 'Trough To Recovery'] = n_recovery

    n_top = min(n_top, len(df_tk_drawdowns))

    ascending = True if sort_by in cols_float + cols_str else False
    df_tk_drawdowns = df_tk_drawdowns.sort_values(by=sort_by, ascending=ascending)
    df_tk_drawdowns = df_tk_drawdowns.reset_index(drop=True)

    df_tk_deepest_drawdowns = df_tk_drawdowns.sort_values(by='% Drawdown', ascending=True)
    df_tk_deepest_drawdowns = df_tk_deepest_drawdowns.reset_index(drop=True)[:n_top]
 
    df_tk_longest_drawdowns = df_tk_drawdowns.sort_values(by='Total Length', ascending=False)
    df_tk_longest_drawdowns = df_tk_longest_drawdowns.reset_index(drop=True)[:n_top]

    # Convert output to strings

    for idx in df_tk_drawdowns.index:
        df_tk_drawdowns_str.loc[idx, 'Peak'] = f"{df_tk_drawdowns.loc[idx, 'Peak']:.2f}"
        df_tk_drawdowns_str.loc[idx, 'Trough'] = f"{df_tk_drawdowns.loc[idx, 'Trough']:.2f}"
        df_tk_drawdowns_str.loc[idx, 'Peak Date'] = f"{df_tk_drawdowns.loc[idx, 'Peak Date'].date()}"
        df_tk_drawdowns_str.loc[idx, 'Trough Date'] = f"{df_tk_drawdowns.loc[idx, 'Trough Date'].date()}"
        df_tk_drawdowns_str.loc[idx, 'Recovery Date'] = f"{df_tk_drawdowns.loc[idx, 'Recovery Date'].date()}"
        df_tk_drawdowns_str.loc[idx, '% Drawdown'] = f"{(df_tk_drawdowns.loc[idx, '% Drawdown']):.2f}"
        df_tk_drawdowns_str.loc[idx, 'Total Length'] = f"{df_tk_drawdowns.loc[idx, 'Total Length']}"
        df_tk_drawdowns_str.loc[idx, 'Peak To Trough'] = f"{df_tk_drawdowns.loc[idx, 'Peak To Trough']}"
        df_tk_drawdowns_str.loc[idx, 'Trough To Recovery'] = f"{df_tk_drawdowns.loc[idx, 'Trough To Recovery']}"

    for idx in df_tk_deepest_drawdowns.index:
        df_tk_deepest_drawdowns_str.loc[idx, 'Peak'] = f"{df_tk_deepest_drawdowns.loc[idx, 'Peak']:.2f}"
        df_tk_deepest_drawdowns_str.loc[idx, 'Trough'] = f"{df_tk_deepest_drawdowns.loc[idx, 'Trough']:.2f}"
        df_tk_deepest_drawdowns_str.loc[idx, 'Peak Date'] = f"{df_tk_deepest_drawdowns.loc[idx, 'Peak Date'].date()}"
        df_tk_deepest_drawdowns_str.loc[idx, 'Trough Date'] = f"{df_tk_deepest_drawdowns.loc[idx, 'Trough Date'].date()}"
        df_tk_deepest_drawdowns_str.loc[idx, 'Recovery Date'] = f"{df_tk_deepest_drawdowns.loc[idx, 'Recovery Date'].date()}"
        df_tk_deepest_drawdowns_str.loc[idx, '% Drawdown'] = f"{(df_tk_deepest_drawdowns.loc[idx, '% Drawdown']):.2f}"
        df_tk_deepest_drawdowns_str.loc[idx, 'Total Length'] = f"{df_tk_deepest_drawdowns.loc[idx, 'Total Length']}"
        df_tk_deepest_drawdowns_str.loc[idx, 'Peak To Trough'] = f"{df_tk_deepest_drawdowns.loc[idx, 'Peak To Trough']}"
        df_tk_deepest_drawdowns_str.loc[idx, 'Trough To Recovery'] = f"{df_tk_deepest_drawdowns.loc[idx, 'Trough To Recovery']}"

    for idx in df_tk_longest_drawdowns.index:
        df_tk_longest_drawdowns_str.loc[idx, 'Peak'] = f"{df_tk_longest_drawdowns.loc[idx, 'Peak']:.2f}"
        df_tk_longest_drawdowns_str.loc[idx, 'Trough'] = f"{df_tk_longest_drawdowns.loc[idx, 'Trough']:.2f}"
        df_tk_longest_drawdowns_str.loc[idx, 'Peak Date'] = f"{df_tk_longest_drawdowns.loc[idx, 'Peak Date'].date()}"
        df_tk_longest_drawdowns_str.loc[idx, 'Trough Date'] = f"{df_tk_longest_drawdowns.loc[idx, 'Trough Date'].date()}"
        df_tk_longest_drawdowns_str.loc[idx, 'Recovery Date'] = f"{df_tk_longest_drawdowns.loc[idx, 'Recovery Date'].date()}"
        df_tk_longest_drawdowns_str.loc[idx, '% Drawdown'] = f"{(df_tk_longest_drawdowns.loc[idx, '% Drawdown']):.2f}"
        df_tk_longest_drawdowns_str.loc[idx, 'Total Length'] = f"{df_tk_longest_drawdowns.loc[idx, 'Total Length']}"
        df_tk_longest_drawdowns_str.loc[idx, 'Peak To Trough'] = f"{df_tk_longest_drawdowns.loc[idx, 'Peak To Trough']}"
        df_tk_longest_drawdowns_str.loc[idx, 'Trough To Recovery'] = f"{df_tk_longest_drawdowns.loc[idx, 'Trough To Recovery']}"

    drawdown_data = {
        'Drawdown Stats': df_tk_drawdowns,
        'Deepest Drawdowns': df_tk_deepest_drawdowns,
        'Longest Drawdowns': df_tk_longest_drawdowns,
        'Drawdown Stats Str': df_tk_drawdowns_str,
        'Deepest Drawdowns Str': df_tk_deepest_drawdowns_str,
        'Longest Drawdowns Str': df_tk_longest_drawdowns_str
    }

    return drawdown_data

# Assume user's input for the number of top drawdowns
n_top = 6

tk = 'AAPL'
# sort_by = 'Total Length'
sort_by = '% Drawdown'
# sort_by = 'Peak Date'
drawdown_data = summarize_tk_drawdowns(df_adj_close, tk, sort_by, n_top)
df_tk_drawdowns = drawdown_data['Drawdown Stats']
df_tk_deepest_drawdowns = drawdown_data['Deepest Drawdowns']
df_tk_longest_drawdowns = drawdown_data['Longest Drawdowns']  # make it largest total period 
df_tk_drawdowns_str = drawdown_data['Drawdown Stats Str']
df_tk_deepest_drawdowns_str = drawdown_data['Deepest Drawdowns Str']
df_tk_longest_drawdowns_str = drawdown_data['Longest Drawdowns Str']  # make it largest total period 

display(df_tk_drawdowns_str)
display(df_tk_deepest_drawdowns_str)
display(df_tk_longest_drawdowns_str)

# print(df_tk_drawdowns.dtypes)

,Peak,Trough,% Drawdown,Peak Date,Trough Date,Recovery Date,Total Length,Peak To Trough,Trough To Recovery
0,197.36,164.59,-16.61,2023-12-14,2024-04-19,2024-06-11,123,87,36
1,234.55,206.99,-11.75,2024-07-16,2024-08-06,2024-09-13,44,16,28
2,179.79,166.04,-7.65,2023-10-12,2023-10-26,2023-11-07,19,11,8
3,178.16,169.56,-4.82,2023-09-19,2023-09-27,2023-10-11,17,7,10
4,216.42,207.25,-4.24,2024-06-17,2024-06-21,2024-07-01,10,4,6
5,232.71,227.31,-2.32,2024-07-10,2024-07-11,2024-07-15,4,2,2
6,194.97,192.45,-1.29,2023-12-08,2023-12-11,2023-12-13,4,2,2
7,190.73,188.65,-1.09,2023-11-20,2023-11-29,2023-12-05,11,7,4


,Peak,Trough,% Drawdown,Peak Date,Trough Date,Recovery Date,Total Length,Peak To Trough,Trough To Recovery
0,197.36,164.59,-16.61,2023-12-14,2024-04-19,2024-06-11,123,87,36
1,234.55,206.99,-11.75,2024-07-16,2024-08-06,2024-09-13,44,16,28
2,179.79,166.04,-7.65,2023-10-12,2023-10-26,2023-11-07,19,11,8
3,178.16,169.56,-4.82,2023-09-19,2023-09-27,2023-10-11,17,7,10
4,216.42,207.25,-4.24,2024-06-17,2024-06-21,2024-07-01,10,4,6
5,232.71,227.31,-2.32,2024-07-10,2024-07-11,2024-07-15,4,2,2


,Peak,Trough,% Drawdown,Peak Date,Trough Date,Recovery Date,Total Length,Peak To Trough,Trough To Recovery
0,197.36,164.59,-16.61,2023-12-14,2024-04-19,2024-06-11,123,87,36
1,234.55,206.99,-11.75,2024-07-16,2024-08-06,2024-09-13,44,16,28
2,179.79,166.04,-7.65,2023-10-12,2023-10-26,2023-11-07,19,11,8
3,178.16,169.56,-4.82,2023-09-19,2023-09-27,2023-10-11,17,7,10
4,190.73,188.65,-1.09,2023-11-20,2023-11-29,2023-12-05,11,7,4
5,216.42,207.25,-4.24,2024-06-17,2024-06-21,2024-07-01,10,4,6


In [16]:
from mapping_plot_attributes import theme_style

# theme = 'light'
theme = 'dark'
style = theme_style[theme]
template = style['template']

plotly_layout = go.Layout(template=template)
# plotly_layout = go.Layout(template='plotly_white')
print(plotly_layout.template.layout.xaxis)

top_by = 'depth'  # or 'length' - user input
# top_by = 'length'
top_by_color = style['red_color']
#op_by_color = 'red'  #style['red_color']

show_trough_to_recovery = False  # based on user input, force it to True if top_by == 'length'
# show_trough_to_recovery = True

# alpha_min, alpha_max = 0.1, 1  # colour intensity between 0 and 1
if theme == 'dark':
    alpha_min, alpha_max = 0.1, 0.5  # max intensity covers the grid
else:
    alpha_min, alpha_max = 0.1, 0.8  # max intensity covers the grid

if top_by == 'depth':
    top_list = list(df_tk_deepest_drawdowns['% Drawdown'])
    top_cmap = map_values(top_list, alpha_min, alpha_max, ascending=True)
else:
    top_list = list(df_tk_longest_drawdowns['Total Length'])
    top_cmap = map_values(top_list, alpha_min, alpha_max, ascending=False)


tk = 'AAPL'

title_font_size = 32

fig = make_subplots(rows = 1, cols = 1)
df_tk = df_adj_close[tk]
min_y = min(df_tk)
max_y = max(df_tk)
y_min, y_max = set_axis_limits(min_y, max_y)

x_min = str(df_tk.index.min().date())
x_max = str(df_tk.index.max().date())

print(x_min, x_max)
n_ticks_max = 48

if top_by == 'depth':
    top_drawdowns = df_tk_deepest_drawdowns
    top_drawdowns_str = df_tk_deepest_drawdowns_str
else:
    top_drawdowns = df_tk_longest_drawdowns
    top_drawdowns_str = df_tk_longest_drawdowns_str   

n_top = min(n_top, len(top_drawdowns))

if show_trough_to_recovery | (top_by == 'length'):
    zip_drawdown_parameters = zip(
        top_drawdowns_str.index,
        top_drawdowns_str['Peak Date'],
        top_drawdowns_str['Recovery Date'],
        top_drawdowns['% Drawdown'],
        top_drawdowns['Total Length']
    )
    title_drawdowns = f'{tk} {n_top} Top Drawdowns by {top_by.capitalize()} - Peak To Recovery'    
else:
    zip_drawdown_parameters = zip(
        top_drawdowns_str.index,
        top_drawdowns_str['Peak Date'],
        top_drawdowns_str['Trough Date'],
        top_drawdowns['% Drawdown'],
        top_drawdowns['Peak To Trough']  # This corresponds to the width of the Peak-To-Trough band
    )
    title_drawdowns = f'{tk} {n_top} Top Drawdowns by {top_by.capitalize()} - Peak To Trough'

fig.add_trace(
    go.Scatter(
        x = df_adj_close.index.astype(str),
        y = df_tk,
        line = dict(color = style['basecolor']),
        line_width = 2,
        name = 'Adjusted Close',
        showlegend = True
    )
)
for i, x1, x2, depth, length in zip_drawdown_parameters:
    
    if top_by == 'depth':
        alpha_deepest = top_cmap[depth]
        name = f'{depth:.1f}%, {length}d'
    else:
        alpha_deepest = top_cmap[length]
        name = f'{length}d, {depth:.1f}%'
    
    fig.add_vline(
        x1,
        line_color = 'brown',
        layer = 'below'
    )
    fig.add_vline(
        x2,
        line_color = 'brown',
        layer = 'below'
    )
    fig.add_trace(
        go.Scatter(
            x = [x1, x2, x2, x1, x1],
            y = [y_min, y_min, y_max, y_max, y_min],
            mode = 'lines',
            line_width = 0,
            fill = 'toself',
            fillcolor = top_by_color,
            opacity = alpha_deepest,
            name = name
        )
    )

# Add here to make sure it's on top of other layers
fig.add_trace(
    go.Scatter(
        x = df_adj_close.index.astype(str),
        y = df_tk,
        line = dict(color = style['basecolor']),
        line_width = 2,
        name = 'Adjusted Close',
        showlegend = False
    )
)

# Add plot border
fig.add_shape(
    type = 'rect',
    xref = 'x',  # use 'x' because of the seconday axis - 'paper' does not work correctly
    yref = 'paper',
    x0 = x_min,
    x1 = x_max,
    y0 = 0,
    y1 = 1,
    line_color = style['x_linecolor'],
    line_width = 0.3
)

# Update layout and axes
fig.update_layout(
    width = 1450,
    height = 750,
    xaxis_rangeslider_visible = False,
    template = template,
    yaxis_title = f'Adjusted Close',
    title = dict(
        text = title_drawdowns,
        font_size = title_font_size,
        y = 0.95,
        x = 0.45,
        xanchor = 'center',
        yanchor = 'top'
    )
)
fig.update_xaxes(
    type = 'category',
    showgrid = True,
    gridcolor = style['x_gridcolor'],
    nticks = n_ticks_max,
    tickangle = -90,
    ticks = 'outside',
    ticklen = 8,
    ticklabelshift = 5,  # not working
    ticklabelstandoff = 10  # not working
)
fig.update_yaxes(
    range = (y_min, y_max),
    showgrid = True,
    gridcolor = style['y_gridcolor'],
    ticks = 'outside',
    ticklen = 8,
    ticklabelshift = 5,  # not working
    ticklabelstandoff = 20  # not working
)

fig.show()

# For details of default styles in each plotle template see
# https://github.com/plotly/plotly.py/blob/master/packages/python/plotly/templategen/definitions.py

layout.XAxis({
    'automargin': True,
    'gridcolor': '#283442',
    'linecolor': '#506784',
    'ticks': '',
    'title': {'standoff': 15},
    'zerolinecolor': '#283442',
    'zerolinewidth': 2
})
2023-09-13 2024-09-13


NOTE: Must check if the number of drawdowns, user-selected or default, actually exists within the time period under consideration - DONE!